# Geocoding Police Fire Departments and Mapping to CSI360 Customer Locations

- Import and data from CSI360 Police and Fire List.
- Geocode addresses to obtain latitude and longitude coordinates.
- Insert latitude and longitude columns into data file.
- Check each department location and identify if it is within 100 miles of the customer locations.
- Separate the dataframe for departments within 100 miles for each customer location.
- Export the data to CSV files for further analysis and reporting.

In [1]:
# Dependencies and Setup
import pandas as pd
from geopy.geocoders import GoogleV3
from geopy.distance import geodesic
# import folium # Use to create html map
# from folium import plugins # Use to create html map
from dotenv import load_dotenv
import os
from ratelimit import limits, sleep_and_retry
import time


load_dotenv()
google_api_key = 'MY API' # Hard coding works

# os.environ.get('GOOGLE_MAPS_API_KEY') Using this causes problems

In [2]:
# Store filepath in a variable
lefd_1_data = 'resources/data/lefd_1_data.csv'
csi360_customers = 'resources/data/csi360_customers.csv'

# Read each of the respective files (police, fire, agency_n, agency_addrs) and store into Pandas dataframe
lefd_1_data_df = pd.read_csv(lefd_1_data)
csi360_customers_df = pd.read_csv(csi360_customers)

# Convert the 'zip' column to integer, then back to string to remove decimals
csi360_customers_df['zip'] = csi360_customers_df['zip'].fillna("").apply(lambda x: str(int(float(x))) if x != "" else "")

display(lefd_1_data_df.head(), csi360_customers_df.head())

,agency_name,agency_type,sworn_active_persnl,hq_addr1,hq_addr2,hq_city,hq_state,hq_zip,addr1,addr2,po_box,city,state,county,zip,hq_ph,hq_fax,org_type,website,fips
0,New York City Police Department,Local police department,36023,,,,,,"One Police Plaza, Room 1400",,,New York,NY,New York,10038,,,,,36061
1,Chicago Police Dept,Local police department,13354,,,,,,3510 S Michigan,,,Chicago,IL,Cook,60653,,,,,17031
2,Los Angeles Police Department,Local police department,9727,,,,,,150 N. Los Angeles Street,,,Los Angeles,CA,Los Angeles,90012,,,,,6037
3,Los Angeles County Sheriff's Office,Sheriff's office,9461,,,,,,Unit 1,4700 W. Ramona Blvd.,,Monterey Park,CA,Los Angeles,91754,,,,,6037
4,California Highway Patrol,Primary state law enforcement agency,7202,,,,,,2555 First Ave,,,Sacramento,CA,Sacramento,95818,,,,,6067


,company,contact,zip
0,Berkeley County Sheriff's Office,Lt. Geno Alterio,29461
1,Blytheville Police Department,Vanessa Stewart,72315
2,Cade Fletcher Sheriff's Office,Cade Fletcher,
3,City of Hagertown,John R Crist,
4,City of Yoakum,Michael Pierman,


In [3]:
## Geocode CSI360 customer zip codes

In [4]:
# Initialize geolocator. 
geolocator = GoogleV3(api_key=google_api_key)

In [5]:
# Function to get geocode based on zip code

def get_lat_long(zip_code):
    if zip_code == "":  # Skip empty ZIP codes
        return None, None
    try:
        # Use the ZIP code directly to get the location
        location = geolocator.geocode(zip_code)
        if location:
            return location.latitude, location.longitude
        else:
            print(f"Geocoding failed for ZIP code: {zip_code}")  # Log failed geocodes
            return None, None
    except Exception as e:
        print(f"Error geocoding {zip_code}: {e}")
        return None, None

# Apply the geocode function to get Latitude and Longitude for each row in the cleaned DataFrame
csi360_customers_df['Latitude'], csi360_customers_df['Longitude'] = zip(*csi360_customers_df['zip'].apply(get_lat_long))

# Fill NaN values in 'Latitude' and 'Longitude' with empty strings
csi360_customers_df['Latitude'] = csi360_customers_df['Latitude'].fillna("")
csi360_customers_df['Longitude'] = csi360_customers_df['Longitude'].fillna("")

# Print the first few rows to verify
print(csi360_customers_df.head())


                            company           contact    zip   Latitude  \
0  Berkeley County Sheriff's Office  Lt. Geno Alterio  29461  33.146521   
1     Blytheville Police Department   Vanessa Stewart  72315  35.893377   
2    Cade Fletcher Sheriff's Office     Cade Fletcher                     
3                 City of Hagertown      John R Crist                     
4                    City of Yoakum   Michael Pierman                     

   Longitude  
0 -79.986407  
1 -89.906631  
2             
3             
4             


## Geocode Colorado Fire Deptartment Addresses


In [6]:
# Function to get latitude and longitude from address
# Set rate limit to 25 requests per second
# Google geocoding API allows around 50 queries per second, I've adjusted to 25 to be safe
RATE_LIMIT = 25  # requests per second
TIME_PERIOD = 1  # time period in seconds

# Function to get latitude and longitude from address with rate limiter
@sleep_and_retry
@limits(calls=RATE_LIMIT, period=TIME_PERIOD)
def get_lat_long(address):
    try:
        location = geolocator.geocode(address)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except Exception as e:
        print(f"Error geocoding {address}: {e}")
        return None, None

In [7]:
# Test with a known address
location = geolocator.geocode("1600 Pennsylvania Ave NW, Washington, DC 20500")
print(location.latitude, location.longitude)

38.89764 -77.0363948


In [8]:
# Combine address fields and get latitude/longitude
lefd_1_data_df['Full_Address'] = lefd_1_data_df['addr1'] + ', ' + lefd_1_data_df['city'] + ', ' + lefd_1_data_df['state'] + ' ' + lefd_1_data_df['zip'].astype(str)
lefd_1_data_df['Latitude'], lefd_1_data_df['Longitude'] = zip(*lefd_1_data_df['Full_Address'].apply(get_lat_long))
print(lefd_1_data_df.head())

                           agency_name                           agency_type  \
0      New York City Police Department               Local police department   
1                  Chicago Police Dept               Local police department   
2        Los Angeles Police Department               Local police department   
3  Los Angeles County Sheriff's Office                      Sheriff's office   
4            California Highway Patrol  Primary state law enforcement agency   

   sworn_active_persnl hq_addr1 hq_addr2 hq_city hq_state hq_zip  \
0                36023                                             
1                13354                                             
2                 9727                                             
3                 9461                                             
4                 7202                                             

                         addr1                 addr2  ...       county    zip  \
0  One Police Plaza, Room 140